# 🧩 HRM Sudoku-Extreme 1 k Demo
**Google Colab PRO (High-RAM) + T4 GPU – single-GPU reproduction of the paper’s 1 k-shot run.**  
Runtime: ~50 min on A100-high-ram, ~55 min on T4-high-ram.

In [3]:
#@title 0️⃣ Check GPU
!nvidia-smi

Sat Aug  2 12:45:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
#@title 1️⃣ One-liner installs (CUDA 12.6 + PyTorch 2.4 + Flash-Attn 2)
import os, subprocess, sys
def run(cmd):
    try:
        subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("Command failed:", e.cmd)
        print("Return code:", e.returncode)
        print("Output (stdout):", e.stdout)
        print("Error (stderr):", e.stderr)
        raise # Re-raise the exception after printing

# PyTorch 2.7 + CUDA 12.6 wheels
run("pip install torch==2.7.0+cu126 --index-url https://download.pytorch.org/whl/cu126 --no-cache-dir --force-reinstall")
run("pip install torchvision==0.22.1+cu126 --index-url https://download.pytorch.org/whl/cu126 --no-cache-dir --force-reinstall")
run("pip install torchaudio==2.7.0+cu126 --index-url https://download.pytorch.org/whl/cu126 --no-cache-dir --force-reinstall")


# Ninja + setuptools for compilation
run("pip install packaging ninja wheel setuptools setuptools-scm")

# Flash-Attention 2 (works on T4/A100)
run("pip install flash-attn --no-build-isolation")

In [20]:
# already inside /content/HRM
git config --global url."https://github.com/".insteadOf "git@github.com:"
git submodule update --init --recursive --depth=1 || true   # ignore ARC failures

SyntaxError: invalid syntax (ipython-input-2755294961.py, line 2)

In [21]:
!ls -l dataset/

total 36
-rw-r--r-- 1 root root 10084 Aug  2 13:59 build_arc_dataset.py
-rw-r--r-- 1 root root  4461 Aug  2 13:59 build_maze_dataset.py
-rw-r--r-- 1 root root  5753 Aug  2 13:59 build_sudoku_dataset.py
-rw-r--r-- 1 root root  1381 Aug  2 13:59 common.py
drwxr-xr-x 5 root root  4096 Aug  2 13:59 raw-data


In [22]:
!python dataset/build_sudoku_dataset.py \
  --output-dir data/sudoku-extreme-1k-aug-1000 \
  --subsample-size 1000 \
  --num-aug 1000

Traceback (most recent call last):
  File "/content/HRM/dataset/build_sudoku_dataset.py", line 7, in <module>
    from argdantic import ArgParser
ModuleNotFoundError: No module named 'argdantic'


In [23]:
!pip install -r requirements.txt   # already done upstream? safe to re-run

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
  Created wheel for adam-atan2: filename=adam_atan2-0.0.3-cp311-cp311-linux_x86_64.whl size=196238 sha256=a3cf4847712d8c8c94a462793a1eaf92e7c44a0b5ef079dd130ecdf46dbf6182
  Stored in directory: /root/.cache/pip/wheels/43/58/2a/9b3bd25c65b754ff4182332021a5ffff4fe68382acae55520f
Successfully built adam-atan2


## 4️⃣ Build the Sudoku-Extreme 1 k dataset  
This is exactly the same as the paper’s `subsample-size 1000 --num-aug 1000`.

In [24]:
#@title 4️⃣ Build dataset (~30 s)
run("python dataset/build_sudoku_dataset.py --output-dir data/sudoku-extreme-1k-aug-1000 --subsample-size 1000 --num-aug 1000")
!ls data/sudoku-extreme-1k-aug-1000

identifiers.json  test	train


## 5️⃣ Train (single GPU, small batch)
We halve the batch size (192 instead of 384) to fit T4 16 GB.  
The run will auto-log to Weights & Biases if you’re logged in (`wandb login`).

In [36]:
from google.colab import files
files.view('/content/HRM/pretrain.py')

<IPython.core.display.Javascript object>

In [37]:
%%bash
cd /content/HRM
python pretrain.py \
  data_path=data/sudoku-extreme-1k-aug-1000 \
  epochs=2000 \
  eval_interval=500 \
  global_batch_size=192 \
  lr=7e-5 \
  puzzle_emb_lr=7e-5 \
  weight_decay=1.0 \
  puzzle_emb_weight_decay=1.0

[Rank 0, World Size 1]: Epoch 0


  0%|          | 0/10416 [00:00<?, ?it/s]W0802 14:16:56.370000 27561 torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:2178: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:2178: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:2178: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:2178: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:2178: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warn

CalledProcessError: Command 'b'cd /content/HRM\npython pretrain.py \\\n  data_path=data/sudoku-extreme-1k-aug-1000 \\\n  epochs=2000 \\\n  eval_interval=500 \\\n  global_batch_size=192 \\\n  lr=7e-5 \\\n  puzzle_emb_lr=7e-5 \\\n  weight_decay=1.0 \\\n  puzzle_emb_weight_decay=1.0\n'' returned non-zero exit status 1.

## 6️⃣ Evaluate
After training finishes (~step 1500) we run the built-in exact-accuracy evaluator.

In [ ]:
#@title 6️⃣ Evaluate last checkpoint
ckpt_path = !ls -t checkpoints/*/ckpt.pt | head -1
ckpt_path = ckpt_path[0]
print("Evaluating", ckpt_path)
run(f"python evaluate.py checkpoint={ckpt_path}")

## 7️⃣ Show one solved grid
We decode the first validation sample back to a human-readable Sudoku.

In [ ]:
#@title 7️⃣ Pretty print a solved puzzle
from src.utils.sudoku import Sudoku
import torch

ckpt = torch.load(ckpt_path, map_location="cpu")
model = ckpt["model"]
model.eval()

from src.data.sudoku_dataset import SudokuDataset
ds = SudokuDataset("data/sudoku-extreme-1k-aug-1000", split="val")
sample = ds[0]

with torch.no_grad():
    logits = model(sample["input_ids"].unsqueeze(0).cuda())
pred = logits.argmax(-1).cpu()

print("Input puzzle:\n", Sudoku(sample["input_ids"].view(9,9)).grid)
print("Model solution:\n", Sudoku(pred.view(9,9)).grid)
print("Target:\n", Sudoku(sample["target"].view(9,9)).grid)

## 8️⃣ Save checkpoint to Drive (optional)
Mount your Drive and copy the 120 MB checkpoint so others can load it instantly.

In [ ]:
#@title 8️⃣ Mount Drive & save
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/hrm_sudoku1k_t4"
run(f"mkdir -p {save_dir}")
run(f"cp -r checkpoints {save_dir}")
print("Checkpoint saved to", save_dir)

# Task
Explain the error in the selected code. If possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error.

## Summary:

### Data Analysis Key Findings

*   The initial attempt to clone the repository failed because a directory with the same name already existed.
*   The existing directory was successfully removed, allowing the repository to be cloned without initializing submodules.
*   The `.gitmodules` file was read, and SSH URLs were successfully replaced with HTTPS URLs.
*   The modified content of the `.gitmodules` file was written back to the file.
*   The submodules were successfully initialized and updated using the modified configuration.